In [75]:
import mailparser
from os import listdir
from os.path import isfile, join
import pandas as pd
import email

First I will get the list of each mail directory in the dataset. There should be 150 different people in the dataset. 

In [76]:
mail_dir = '../data/maildir/'
mailboxes = listdir(mail_dir)
print(len(mailboxes))

150


Temporarily start with one person. This will be for each directoy later on. 

In [77]:
mailbox = mailboxes[0]
inbox = listdir(mail_dir + mailbox)
print(inbox)

['sent_items', 'inbox', 'discussion_threads', 'notes_inbox', 'deleted_items', 'sent', 'all_documents']


We need to start with a list, that will represent each row of the dataframe. Each item in the row is a dict of parsed email values. 

In [78]:
enron_email_list = []

Now put each email into a pandas dataframe. process_email will create a 1 row dataframe to append to the full dataframe. 

In [81]:
def process_email(email_path, employee, folder, email_name):
    mail = mailparser.parse_from_file(email_path)
    
    #The following strips out the forwarded portion of the email
    fwds = []
    
    full_body = email.message_from_string(mail.body)
    
    #Seperate the body of the email
    if full_body.is_multipart():
        for payload in full_body.get_payload():
            fwds.append(payload.get_payload())
        mail_body = fwds[0]
        fwds.remove(0)
    else:
        mail_body = full_body.get_payload() 
    
    email_dict = {
                "Employee" : employee,
                "Folder" : folder,
                "Email-Name" : email_name,
                "Body" : mail_body,
                "Fwd_chain": fwds
    }
    #Put rest of header into the row. 
    email_dict.update(mail.headers)
    enron_email_list.append(email_dict)
    

In [82]:
#process_email(full_path, mailbox, folder[0], single_email[0])

for folder in inbox:
    path = mail_dir + mailbox + "/" + folder
    emails = listdir(path)
    for single_email in emails:
        full_path = path + "/" + single_email
        process_email(full_path, mailbox, folder, single_email)
enron_email_list_df = pd.DataFrame(enron_email_list)
enron_email_list_df

Bcc  \
0      NaN                                                                                                                           
1      NaN                                                                                                                           
2      NaN                                                                                                                           
3      NaN                                                                                                                           
4      NaN                                                                                                                           
5      NaN                                                                                                                           
6      NaN                                                                                                                           
7      NaN                                                                                                                           
8      NaN                                                                                                                           
9      NaN                                                                                                                           
10     ralee@rmi.net, pederleifkjeseth@hotmail.com, scott.morrill@grubb-ellis.com                                                    
11     NaN                                                                                                                           
12     NaN                                                                                                                           
13     NaN                                                                                                                           
14     NaN                                                                                                                           
15     NaN                                                                                                                           
16     NaN                                                                                                                           
17     chad.landry@enron.com                                                                                                         
18     NaN                                                                                                                           
19     NaN                                                                                                                           
20     NaN                                                                                                                           
21     NaN                                                                                                                           
22     NaN                                                                                                                           
23     NaN                                                                                                                           
24     NaN                                                                                                                           
25     NaN                                                                                                                           
26     NaN                                                                                                                           
27     NaN                                                                                                                           
28     NaN                                                                                                                           
29     NaN                                                                                                 

In [83]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns',500)
enron_email_list_df[['Body']]

Body
0      yep.  looks like i found the formula.\n\n -----Original Message-----\nFrom: \tErin Richardson <erichardson@sarofim.com>@ENRON  \nSent:\tFriday, October 12, 2001 10:18 AM\nTo:\tLenhart, Matthew\nSubject:\tRE:\n\nare you having a better day today?\n\n-----Original Message-----\nFrom: Matthew.Lenhart@enron.com [mailto:Matthew.Lenhart@enron.com]\nSent: Thursday, October 11, 2001 5:15 PM\nTo: erichardson@sarofim.com\nSubject: RE:\n\n\nyou are so mean.  why can't you be sweet like me?\n\n    -----Original Message-----\n   From:   Erin Richardson <erichardson@sarofim.com>@ENRON\n   Sent:   Thursday, October 11, 2001 4:58 PM\n   To:     Lenhart, Matthew\n   Subject:  RE:\n\n   whatever, i know you were kidding, so was i.\n\n   -----Original Message-----\n   From: Matthew.Lenhart@enron.com [mailto:Matthew.Lenhart@enron.com]\n   Sent: Thursday, October 11, 2001 4:58 PM\n   To: erichardson@sarofim.com\n   Subject: RE:\n\n\n   just kidding.  you need to relax a little.\n\n       -----Original Message-----\n      From:   Erin Richardson <erichardson@sarofim.com>@ENRON\n      Sent:   Thursday, October 11, 2001 4:50 PM\n      To:     Lenhart, Matthew\n      Subject:  RE:\n\n      you are a jerk\n\n      -----Original Message-----\n      From: Matthew.Lenhart@enron.com [mailto:Matthew.Lenhart@enron.com]\n      Sent: Thursday, October 11, 2001 4:51 PM\n      To: erichardson@sarofim.com\n      Subject: RE:\n\n\n      i am going to the strip club.  do you want to meet me there?  i will\n   get\n      you a lap dance.\n\n          -----Original Message-----\n         From:   Erin Richardson <erichardson@sarofim.com>@ENRON\n         Sent:   Thursday, October 11, 2001 3:38 PM\n         To:     Lenhart, Matthew\n         Subject:  RE:\n\n          you'll be fine\n\n         -----Original Message-----\n         From: Matthew.Lenhart@enron.com [mailto:Matthew.Lenhart@enron.com]\n         Sent: Thursday, October 11, 2001 3:34 PM\n         To: erichardson@sarofim.com\n         Subject: RE:\n\n\n         i resent that too\n\n             -----Original Message-----\n            From:   Erin Richardson <erichardson@sarofim.com>@ENRON\n            Sent:   Thursday, October 11, 2001 3:30 PM\n            To:     Lenhart, Matthew\n            Subject:  RE:\n\n            why? it's true.  i resent the fact that you can't be honest\n   with\n      me\n\n            -----Original Message-----\n            From: Matthew.Lenhart@enron.com\n   [mailto:Matthew.Lenhart@enron.com]\n            Sent: Thursday, October 11, 2001 3:29 PM\n            To: erichardson@sarofim.com\n            Subject: RE:\n\n\n            i resent that last comment\n\n                -----Original Message-----\n               From:   Erin Richardson <erichardson@sarofim.com>@ENRON\n               Sent:   Thursday, October 11, 2001 3:25 PM\n               To:     Lenhart, Matthew\n               Subject:  RE:\n\n               exactly, see, worthless.\n\n               -----Original Message-----\n               From: Matthew.Lenhart@enron.com\n      [mailto:Matthew.Lenhart@enron.com]\n               Sent: Thursday, October 11, 2001 3:22 PM\n               To: erichardson@sarofim.com\n               Subject: RE:\n\n\n               i am gonna buy you a bmw\n\n                   -----Original Message-----\n                  From:   Erin Richardson <erichardson@sarofim.com>@ENRON\n                  Sent:   Thursday, October 11, 2001 3:19 PM\n                  To:     Lenhart, Matthew\n                  Subject:  RE:\n\n                  whatever, from now on i just won't listen to anything you\n      say.\n         i\n               will\n                  consider everything that comes out of your mouth\n   worthless.\n\n                  -----Original Message-----\n                  From: Matthew.Lenhart@enron.com\n         [mailto:Matthew.Lenhart@enron.com]\n                  Sent: Thursday, October 11, 2001 3:07 PM\n                  To: erichardson@sarofim.com\n                 